In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
data = pd.read_csv('points_5.csv')
data.drop('label',axis=1,inplace=True)

In [3]:
data.shape

(128, 132)

In [4]:
# Define LSTM model class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, sequence_length, num_classes):
        super(RNN, self).__init__()
        

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=0.4, batch_first=True)

        # # Attention layer
        # self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=8, batch_first=True)

        self.fc = nn.Linear(hidden_size*sequence_length, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.leaky_relu = nn.ReLU(0.1)
        
    def forward(self, x):
        # # Initialize hidden state and cell state
        # h0 = torch.randn(self.num_layers, x.size(0), self.hidden_size).to(device)
        # c0 = torch.randn(self.num_layers, x.size(0), self.hidden_size).to(device)
    
        # Forward propagate LSTM
        out, _ = self.lstm(x)

        # # Apply attention
        # out, _ = self.attention(out, out, out)

        # print("BEFORE FC")
        # print(out.shape)
        # Decode the hidden state of the last time step :: out.shape = (batch_size, seq_length, hidden_size)
        out = out.contiguous().view(out.size(0), -1)
        # print("AFTER CONversion")
        # print(out.shape)
        out = self.fc(out)
        out = self.leaky_relu(out)
        out = self.fc2(out)
        out = self.leaky_relu(out)
        out = self.fc3(out)
        # print("AFTER FC")
        # print(out.shape)

        # # Apply softmax activation along the sequence dimension
        # out = torch.softmax(out, dim=1)
        
        return out

In [5]:
input_size = 680  # Number of features
hidden_size = 256 # Hidden layer size
num_layers = 1  # Number of LSTM layers
num_classes = 6  # Number of output classes
learning_rate = 0.001  # Learning rate
num_epochs = 10  # Number of epochs
batch_size = 8 # Batch size
sequence_length = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model = RNN(input_size, hidden_size, num_layers, sequence_length, num_classes).to(device) # Replace with your model class
model.load_state_dict(torch.load('models/classify-model.pth'))
model.eval()

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RNN(
  (lstm): LSTM(680, 256, batch_first=True, dropout=0.4)
  (fc): Linear(in_features=2560, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=6, bias=True)
  (leaky_relu): ReLU(inplace=True)
)

In [7]:
from classify_predict import predict



output = predict(data,model)

print(output)

featue_df shape:  (128, 68)
shape of feature_df :  (128, 680)
shape (128, 680)
length of error list 128
0
